In [14]:
import pandas as pd
import numpy as np
import json
import csv

In [2]:
#1.确定这几个数据都是干什么的
swiping_card_dataframe = pd.read_csv('./bus_swiping_card_info1.csv')
card_info_dataframe = pd.read_csv('./card_info.csv')
bus_swip_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
bus_station_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')

In [3]:
#swiping_card_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间
#card_info_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间也是上面的信息，但是时间是去年的。
#用去年的时间的同比数据，去预测今年的同比数据，看是否一致，如果一致，说明经济发展没有变化，如果变大，说明经济好转
#bus_swip_dataframe ：一天的统计信息：各条线路、各个站点的刷卡数量信息
#bus_station_dataframe ： 站点的统计信息：各个站点的经纬度

In [56]:
line_station_number_sequen_dataframe = \
bus_station_dataframe.groupby(['line_identity','station_number']).agg({'station_name':list,'station_identity':list}).reset_index()
line_station_number_sequen_dataframe['length'] = line_station_number_sequen_dataframe['station_identity'].apply(lambda x:len(x))
line_station_number_sequen_dataframe

,line_identity,station_number,station_name,station_identity,length
0,1,20,"[火车站, 展览馆, 新路口, 邮电大楼, 大十字, 喷水池, 云中广场, 六广门, 北京路...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",20
1,2,20,"[火车站, 展览馆, 新路口, 河滨公园, 次南门, 大西门, 紫林庵, 延安西路, 黔灵山...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",20
2,3,29,"[贵州理工学院, 贵黄交警支队, 枫丹白鹭花园, 后坝, 三桥水果市场, 三桥, 三桥北路口...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",29
3,4,44,"[新寨公交枢纽, 南厂路南, 军区干休所, 武岳新村, 军区二招, 南厂路北, 纪念塔, 箭...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",44
4,5,36,"[甘荫塘轨道站, 甘荫塘, 四方河路站, 通银配件城, 皂角井站, 凤凰翠堤, 新发装饰市场...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",36
...,...,...,...,...,...
320,10050,46,"[一八三厂, 毛寨村, 小河平桥, 黄河路南, 黄河路北, 四十四医院, 新村, 四方河, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",46
321,10056,31,"[乐湾国际城, 东风镇, 贵州师范学院, 云上村, 117地质队, 温泉路口, 新添寨, 市...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",31
322,10229,28,"[金阳客站, 商城东路, 宾阳大道, 兴筑西路, 碧海乾图, 黔中一中南, 老阳关, 北京路...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",28
323,10262,24,"[黔中北站, 甲秀北路, 杨家庄, 大关, 鹿冲关公园, 相宝山, 师大, 省公安厅, 宝山...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",24


In [77]:
line_station_number_sequen_dataframe['id_station'] = line_station_number_sequen_dataframe.apply(lambda x:list(zip(x['station_identity'],x['station_name'])),axis=1)

In [81]:
line_station_number_sequen_dataframe

,line_identity,station_number,station_name,station_identity,length,id_station
0,1,20,"[火车站, 展览馆, 新路口, 邮电大楼, 大十字, 喷水池, 云中广场, 六广门, 北京路...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",20,"[(1, 火车站), (2, 展览馆), (3, 新路口), (4, 邮电大楼), (5, ..."
1,2,20,"[火车站, 展览馆, 新路口, 河滨公园, 次南门, 大西门, 紫林庵, 延安西路, 黔灵山...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",20,"[(1, 火车站), (2, 展览馆), (3, 新路口), (4, 河滨公园), (5, ..."
2,3,29,"[贵州理工学院, 贵黄交警支队, 枫丹白鹭花园, 后坝, 三桥水果市场, 三桥, 三桥北路口...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",29,"[(1, 贵州理工学院), (2, 贵黄交警支队), (3, 枫丹白鹭花园), (4, 后坝..."
3,4,44,"[新寨公交枢纽, 南厂路南, 军区干休所, 武岳新村, 军区二招, 南厂路北, 纪念塔, 箭...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",44,"[(1, 新寨公交枢纽), (2, 南厂路南), (3, 军区干休所), (4, 武岳新村)..."
4,5,36,"[甘荫塘轨道站, 甘荫塘, 四方河路站, 通银配件城, 皂角井站, 凤凰翠堤, 新发装饰市场...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",36,"[(1, 甘荫塘轨道站), (2, 甘荫塘), (3, 四方河路站), (4, 通银配件城)..."
...,...,...,...,...,...,...
320,10050,46,"[一八三厂, 毛寨村, 小河平桥, 黄河路南, 黄河路北, 四十四医院, 新村, 四方河, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",46,"[(1, 一八三厂), (2, 毛寨村), (3, 小河平桥), (4, 黄河路南), (5..."
321,10056,31,"[乐湾国际城, 东风镇, 贵州师范学院, 云上村, 117地质队, 温泉路口, 新添寨, 市...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",31,"[(1, 乐湾国际城), (2, 东风镇), (3, 贵州师范学院), (4, 云上村), ..."
322,10229,28,"[金阳客站, 商城东路, 宾阳大道, 兴筑西路, 碧海乾图, 黔中一中南, 老阳关, 北京路...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",28,"[(1, 金阳客站), (2, 商城东路), (3, 宾阳大道), (4, 兴筑西路), (..."
323,10262,24,"[黔中北站, 甲秀北路, 杨家庄, 大关, 鹿冲关公园, 相宝山, 师大, 省公安厅, 宝山...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",24,"[(1, 黔中北站), (2, 甲秀北路), (3, 杨家庄), (4, 大关), (5, ..."


In [85]:
#获取每条线路上乘客上车站所对应的可能的下车站：估计数据量为：
#构建一个字典：
#road_structure = {line_id:{'stations_nums':int,'stations':{station_id:station_name},'all_stations':[]}}
road_line_station_structure_setting={}
for row_index in range(len(line_station_number_sequen_dataframe)):
    line_id = line_station_number_sequen_dataframe[row_index:row_index+1]['line_identity'].values[0]
    stations_nums = line_station_number_sequen_dataframe[row_index:row_index+1]['station_number'].values[0]
    all_stations = line_station_number_sequen_dataframe[row_index:row_index+1]['station_name'].values[0]
    stations={}
    for instance in line_station_number_sequen_dataframe[row_index:row_index+1]['id_station'].values[0]:
        instance_id = instance[0]
        instance_name = instance[1]
        stations[instance_id]=instance_name
    road_line = {'stations_nums':stations_nums,'stations':stations,'all_stations':all_stations}
    road_line_station_structure_setting[line_id] = road_line    

In [86]:
road_line_station_structure_setting

{1: {'stations_nums': 20,
  'stations': {1: '火车站',
   2: '展览馆',
   3: '新路口',
   4: '邮电大楼',
   5: '大十字',
   6: '喷水池',
   7: '云中广场',
   8: '六广门',
   9: '北京路',
   10: '北京西路口',
   11: '黔灵山公园',
   12: '黔灵山公园',
   13: '延安西路',
   14: '紫林庵',
   15: '大西门',
   16: '次南门',
   17: '河滨公园',
   18: '新路口',
   19: '展览馆',
   20: '火车站'},
  'all_stations': ['火车站',
   '展览馆',
   '新路口',
   '邮电大楼',
   '大十字',
   '喷水池',
   '云中广场',
   '六广门',
   '北京路',
   '北京西路口',
   '黔灵山公园',
   '黔灵山公园',
   '延安西路',
   '紫林庵',
   '大西门',
   '次南门',
   '河滨公园',
   '新路口',
   '展览馆',
   '火车站']},
 2: {'stations_nums': 20,
  'stations': {1: '火车站',
   2: '展览馆',
   3: '新路口',
   4: '河滨公园',
   5: '次南门',
   6: '大西门',
   7: '紫林庵',
   8: '延安西路',
   9: '黔灵山公园',
   10: '黔灵山公园',
   11: '北京西路口',
   12: '北京路',
   13: '六广门',
   14: '云中广场',
   15: '喷水池',
   16: '大十字',
   17: '邮电大楼',
   18: '新路口',
   19: '展览馆',
   20: '火车站'},
  'all_stations': ['火车站',
   '展览馆',
   '新路口',
   '河滨公园',
   '次南门',
   '大西门',
   '紫林庵',
   '延安西路',
   '黔灵山公园',
   '黔灵山公园',
   '北京西路口'

In [44]:
bus_station_dataframe

,id,line_identity,line_direction,station_number,station_identity,station_name,longitude,latitude
0,1,1047,1,55,1,碧海商业广场,106.618469,26.633052
1,2,1047,1,55,2,碧水云天,106.617741,26.635156
2,3,1047,1,55,3,新瑞商业中心,106.616733,26.637384
3,4,1047,1,55,4,黎阳家园,106.619175,26.638324
4,5,1047,1,55,5,黔中一中（西）,106.620504,26.630221
...,...,...,...,...,...,...,...,...
12141,12142,752,0,14,10,三岔河,106.802352,26.486220
12142,12143,752,0,14,11,花寨,106.800027,26.494632
12143,12144,752,0,14,12,葫芦坡,106.798085,26.497952
12144,12145,752,0,14,13,白腊山,106.760324,26.517857


In [49]:
bus_station_dataframe[bus_station_dataframe['line_identity']==752].reset_index(drop=True)

,id,line_identity,line_direction,station_number,station_identity,station_name,longitude,latitude
0,12133,752,1,14,1,双龙实验中学,106.759572,26.510484
1,12134,752,1,14,2,白腊山,106.760026,26.517252
2,12135,752,1,14,3,葫芦坡,106.767890,26.521159
3,12136,752,1,14,4,花寨,106.797589,26.498001
4,12137,752,1,14,5,三岔河,106.802645,26.486648
5,12138,752,1,14,6,大土,106.798389,26.476731
6,12139,752,1,14,7,小关山,106.811334,26.463391
7,12140,752,0,14,8,小关山,106.811717,26.463955
8,12141,752,0,14,9,大土,106.797958,26.472821
9,12142,752,0,14,10,三岔河,106.802352,26.486220


In [12]:
station_distance_rank_index_list = np.loadtxt('./station_pair_distance_rank_index.csv',delimiter=',',dtype=int)

In [17]:
# 从文件中读取字典
with open('./id_2_station.json', 'r') as file:
    id_2_station = json.load(file)
# 将字典保存到文件中
with open('./station_2_id.json', 'r') as file:
    station_2_id = json.load(file)

In [20]:
#从card_info_dataframe中找出所有用户一天内的od对信息
card_info_dataframe['custom_day'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[0])
card_info_dataframe['custom_precise_time'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[1])
#sum(card_info_dataframe['partitionday']==card_info_dataframe['custom_day'])
#card_info_dataframe
card_info_dataframe.drop(labels=['custom_time','card_type','consume','data_src','data_load_time','partitionday'],axis=1,inplace=True)

In [29]:
day_0929_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-09-29']
day_0929_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,28,7520,54,2023-09-29,08:59:40
1,0x1FFC5D30D96AE2C4524842A09AA67C44,84,5085,24,2023-09-29,11:08:22
2,0x65F9DAD54040FBE48972B10F3B6337DC,25,6718,4,2023-09-29,08:51:28
3,0x747A26B4727A9A3094A1881F29D01510,208,6655,1,2023-09-29,22:32:48
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,204,4472,46,2023-09-29,19:50:37
...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,52,5054,22,2023-09-29,08:48:38
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,1,4662,6,2023-09-29,10:33:45
377161,0x6D191CC173F9415C1148B434D2F9012E,203,4810,5,2023-09-29,14:57:10
377162,0x7896A07D162530E734F876D16301EFEB,73,5176,23,2023-09-29,15:08:49


In [34]:
#按照刷卡id进行聚合
every_user_all_trips_dataframe = \
day_0929_dataframe.groupby('card_identity').agg({'custom_precise_time':list, \
                                                 'line_identity':list, \
                                                 'station_identity':list, \
                                                'car_identity':list},axis=1).reset_index()
#每天承载18万人出行181190人
#乘坐两程以上的有109927人
#乘坐单程的极有可能是流动，也有可能是别的交通方式回。从前后相依有下一段旅程的来确定od对，对于没有后程的暂时不确定。
every_user_all_trips_dataframe['trip_length'] = every_user_all_trips_dataframe['line_identity'].apply(lambda x:len(x))
#every_user_all_trips_dataframe

In [68]:
trip_great_than_2=every_user_all_trips_dataframe[every_user_all_trips_dataframe['trip_length']!=1].reset_index(drop=True)
#trip_great_than_2
trip_great_than_2['trip'] = trip_great_than_2.apply(lambda x:list(zip(x['custom_precise_time'],x['line_identity'],x['station_identity'])),axis=1)
trip_great_than_2

,card_identity,custom_precise_time,line_identity,station_identity,car_identity,trip_length,trip
0,0x0000719AE3ECECBABAD0263AF940450F,"[19:01:59, 16:55:43]","[40, 46]","[31, 1]","[4483, 3762]",2,"[(19:01:59, 40, 31), (16:55:43, 46, 1)]"
1,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16..."
2,0x000229AC214F69A81F266A521394C628,"[07:13:30, 18:07:56]","[7, 19]","[21, 23]","[5020, 5038]",2,"[(07:13:30, 7, 21), (18:07:56, 19, 23)]"
3,0x00065B8345E25E499BDF9B0632576C87,"[16:26:36, 09:58:48]","[57, 57]","[6, 32]","[7344, 7343]",2,"[(16:26:36, 57, 6), (09:58:48, 57, 32)]"
4,0x00067C25B50FB204998F9955D5692DCD,"[13:42:06, 11:24:18]","[16, 306]","[22, 14]","[4536, 5181]",2,"[(13:42:06, 16, 22), (11:24:18, 306, 14)]"
...,...,...,...,...,...,...,...
109922,0xFFFD09DB2DE6FD874710FA9B56E9A1B4,"[09:42:40, 08:03:14]","[8, 319]","[22, 15]","[4875, 4440]",2,"[(09:42:40, 8, 22), (08:03:14, 319, 15)]"
109923,0xFFFD5320ADB5DC148D136F1494654247,"[12:46:31, 12:16:55]","[35, 9]","[8, 2]","[4881, 3882]",2,"[(12:46:31, 35, 8), (12:16:55, 9, 2)]"
109924,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0..."
109925,0xFFFDF917CDD67471BC901AD187061D38,"[14:03:34, 13:16:22]","[47, 60]","[2, 6]","[5042, 4484]",2,"[(14:03:34, 47, 2), (13:16:22, 60, 6)]"


In [93]:
can_used_od_trips=[]
def find_leave_station(sorted_index_array:np.ndarray,specified_indices:list):
    # 找到指定索引组中在排序数组中的位置
    positions = [sorted_index_array.index(idx) for idx in specified_indices]
    return sorted_index_array[min(positions)]
for row_index in range(len(trip_great_than_2)):
    trip_list = trip_great_than_2[row_index:row_index+1]['trip'].values[0]
    sorted_trips_list = sorted(trip_list,key=lambda x:x[0],reverse=False)
    for trip_index in range(len(sorted_trips_list)-1):
        this_time,this_line,this_station_id = sorted_trips_list[trip_index]
        next_time,next_line,next_station_id = sorted_trips_list[trip_index+1]
        try:
            this_line_station_name = road_line_station_structure_setting[this_line]['stations'][this_station_id]
            this_line_station_id = station_2_id[this_line_station_name]
        
            all_possible_leave_stations = road_line_station_structure_setting[this_line]['all_stations']
            all_possible_leave_stations_index = [station_2_id[station] for station in all_possible_leave_stations]
        
            next_line_station_name = road_line_station_structure_setting[next_line]['stations'][next_station_id]
            next_line_station_id = station_2_id[next_line_station_name]
            distance_2_next_line_station_ndarray  = station_distance_rank_index_list[next_line_station_id]
            leave_station = find_leave_station(distance_2_next_line_station_ndarray.tolist(),all_possible_leave_stations_index)
            od_trip = (this_line_station_id,this_time,this_line,next_line_station_id)
            can_used_od_trips.append(od_trip)
        except:
            continue

In [103]:
#将可用的od对转化为dataframe：这样就可以使用dataframe进行聚类
#can_used_od_trips[0:5]
can_used_od_trips_dataframe = pd.DataFrame(columns=['start_station_id','time','line','end_station_id'])
for trip_index in range(len(can_used_od_trips)):
    start_station_id,time,line,end_station_id=can_used_od_trips[trip_index]
    can_used_od_trips_dataframe.loc[len(can_used_od_trips_dataframe)] = \
    {'start_station_id':start_station_id, \
                                         'time':time, \
                                         'line':line, \
                                         'end_station_id':end_station_id}

In [104]:
len(can_used_od_trips)
#一共有169099可以确定的od对数据

169099

In [107]:
can_used_od_trips_dataframe['hour']= \
can_used_od_trips_dataframe['time'].apply(lambda x:x.split(':')[0])
can_used_od_trips_dataframe['minu']= \
can_used_od_trips_dataframe['time'].apply(lambda x:x.split(':')[1])

In [105]:
can_used_od_trips_dataframe

,start_station_id,time,line,end_station_id
0,1435,16:55:43,46,1262
1,643,09:20:51,204,436
2,436,09:26:05,207,898
3,898,16:29:50,5,406
4,795,07:13:30,7,1416
...,...,...,...,...
169094,1155,15:07:03,229,1155
169095,1155,15:07:13,229,263
169096,263,16:00:55,12,263
169097,1234,13:16:22,60,419


In [124]:
can_used_od_trips_dataframe.to_json('./can_used_od_trips_dataframe.json', orient='records')

In [125]:
# 使用read_json()函数打开JSON文件
test_can_used_od_trips_dataframe = pd.read_json('./can_used_od_trips_dataframe.json', orient='records')

In [126]:
test_can_used_od_trips_dataframe

,start_station_id,time,line,end_station_id,hour,minu
0,1435,16:55:43,46,1262,16,55
1,643,09:20:51,204,436,9,20
2,436,09:26:05,207,898,9,26
3,898,16:29:50,5,406,16,29
4,795,07:13:30,7,1416,7,13
...,...,...,...,...,...,...
169094,1155,15:07:03,229,1155,15,7
169095,1155,15:07:13,229,263,15,7
169096,263,16:00:55,12,263,16,0
169097,1234,13:16:22,60,419,13,16


In [108]:
can_used_od_trips_dataframe

,start_station_id,time,line,end_station_id,hour,minu
0,1435,16:55:43,46,1262,16,55
1,643,09:20:51,204,436,09,20
2,436,09:26:05,207,898,09,26
3,898,16:29:50,5,406,16,29
4,795,07:13:30,7,1416,07,13
...,...,...,...,...,...,...
169094,1155,15:07:03,229,1155,15,07
169095,1155,15:07:13,229,263,15,07
169096,263,16:00:55,12,263,16,00
169097,1234,13:16:22,60,419,13,16


In [122]:
#'hour','minu','line','start_station_id','end_station_id','time'
#可以获取各个站点各个时刻的上车人数，不管是不是同一个线路
od_trips_groupby_id_hour = can_used_od_trips_dataframe.groupby(['start_station_id', \
                                                                'hour']).agg({'line':list, \
                                                                              'end_station_id':list, \
                                                                              'minu':list}).reset_index()
od_trips_groupby_id_hour[od_trips_groupby_id_hour['start_station_id']==0]

,start_station_id,hour,line,end_station_id,minu
0,0,06,"[226, 226]","[345, 1177]","[38, 38]"
1,0,07,"[226, 228]","[28, 345]","[19, 35]"
2,0,08,[226],[846],[58]
3,0,09,[228],[345],[35]
4,0,10,"[226, 228, 228, 228, 228, 228, 228]","[261, 0, 1070, 1218, 128, 0, 345]","[13, 35, 35, 34, 35, 34, 34]"
5,0,11,"[226, 226, 226]","[846, 846, 507]","[50, 50, 20]"
6,0,12,[228],[467],[35]
7,0,16,[226],[0],[45]
8,0,17,[228],[345],[45]
9,0,18,"[226, 226, 226]","[846, 846, 846]","[52, 51, 52]"


In [128]:
#将各个站点的各个图像画出来，
#选择人流量大的站点，看上车人数分布
od_trips_groupby_id_hour['nums'] = od_trips_groupby_id_hour['line'].apply(lambda x:len(x))
od_trips_groupby_id_hour

,start_station_id,hour,line,end_station_id,minu,nums
0,0,06,"[226, 226]","[345, 1177]","[38, 38]",2
1,0,07,"[226, 228]","[28, 345]","[19, 35]",2
2,0,08,[226],[846],[58],1
3,0,09,[228],[345],[35],1
4,0,10,"[226, 228, 228, 228, 228, 228, 228]","[261, 0, 1070, 1218, 128, 0, 345]","[13, 35, 35, 34, 35, 34, 34]",7
...,...,...,...,...,...,...
12189,1448,13,"[1027, 256, 1027, 1027, 232]","[1166, 774, 1251, 139, 122]","[35, 27, 35, 35, 21]",5
12190,1448,14,"[232, 1027, 256, 66]","[147, 1257, 610, 1143]","[01, 33, 19, 22]",4
12191,1448,15,"[66, 232, 66]","[263, 1177, 1357]","[13, 12, 13]",3
12192,1448,16,[1027],[43],[45],1


In [129]:
# 对DataFrame进行groupby并计算每个组的总和
grouped_sum = od_trips_groupby_id_hour.groupby('start_station_id')['nums'].transform('sum')
grouped_sum

0        22
1        22
2        22
3        22
4        22
         ..
12189    47
12190    47
12191    47
12192    47
12193    47
Name: nums, Length: 12194, dtype: int64

In [130]:
# 计算每行的值占总和的比例
od_trips_groupby_id_hour['nums_proportion'] = od_trips_groupby_id_hour['nums'] / grouped_sum

In [133]:
od_trips_groupby_id_4_nums = od_trips_groupby_id_hour.groupby('start_station_id').agg({'nums':list}).reset_index()
od_trips_groupby_id_4_nums

,start_station_id,nums
0,0,"[2, 2, 1, 1, 7, 3, 1, 1, 1, 3]"
1,1,"[6, 7, 6, 6, 4, 1, 3, 3, 1, 1, 1]"
2,2,"[3, 8, 25, 28, 28, 8, 15, 20, 10, 20, 19, 7, 8..."
3,4,"[1, 5, 2, 1, 1, 1, 1, 1, 1]"
4,7,"[34, 32, 34, 54, 38, 23, 27, 34, 15, 11, 27, 1..."
...,...,...
1126,1441,"[1, 1, 1, 1, 1, 1]"
1127,1442,"[2, 1, 1]"
1128,1443,"[2, 13, 7, 3, 19, 11, 4, 4, 8, 7, 11, 12, 4, 1..."
1129,1444,[1]


In [135]:
od_trips_groupby_id_4_nums['sum'] = od_trips_groupby_id_4_nums['nums'].apply(lambda x:sum(x))
od_trips_groupby_id_4_nums

,start_station_id,nums,sum
0,0,"[2, 2, 1, 1, 7, 3, 1, 1, 1, 3]",22
1,1,"[6, 7, 6, 6, 4, 1, 3, 3, 1, 1, 1]",39
2,2,"[3, 8, 25, 28, 28, 8, 15, 20, 10, 20, 19, 7, 8...",213
3,4,"[1, 5, 2, 1, 1, 1, 1, 1, 1]",14
4,7,"[34, 32, 34, 54, 38, 23, 27, 34, 15, 11, 27, 1...",371
...,...,...,...
1126,1441,"[1, 1, 1, 1, 1, 1]",6
1127,1442,"[2, 1, 1]",4
1128,1443,"[2, 13, 7, 3, 19, 11, 4, 4, 8, 7, 11, 12, 4, 1...",109
1129,1444,[1],1


In [142]:
station_id_2_swip_times = list(zip(od_trips_groupby_id_4_nums['start_station_id'], \
                                   od_trips_groupby_id_4_nums['sum']))
station_id_2_swip_times

[(0, 22),
 (1, 39),
 (2, 213),
 (4, 14),
 (7, 371),
 (8, 175),
 (9, 4),
 (10, 5),
 (12, 1),
 (14, 174),
 (15, 43),
 (16, 252),
 (17, 157),
 (18, 20),
 (19, 49),
 (20, 9),
 (21, 37),
 (22, 28),
 (23, 448),
 (24, 188),
 (25, 59),
 (28, 1081),
 (29, 20),
 (30, 272),
 (31, 708),
 (32, 92),
 (33, 495),
 (34, 6),
 (35, 130),
 (36, 3),
 (38, 1),
 (39, 6),
 (40, 71),
 (41, 1),
 (42, 12),
 (43, 20),
 (44, 266),
 (46, 16),
 (47, 1),
 (48, 18),
 (49, 85),
 (50, 23),
 (52, 2),
 (53, 166),
 (54, 34),
 (55, 27),
 (56, 13),
 (57, 1),
 (58, 4),
 (59, 364),
 (60, 516),
 (61, 432),
 (62, 140),
 (65, 24),
 (66, 200),
 (67, 3),
 (68, 44),
 (69, 143),
 (70, 36),
 (71, 12),
 (72, 53),
 (73, 85),
 (74, 6),
 (75, 94),
 (76, 22),
 (78, 167),
 (79, 192),
 (80, 91),
 (81, 39),
 (82, 34),
 (83, 118),
 (84, 312),
 (85, 139),
 (86, 37),
 (87, 902),
 (88, 748),
 (90, 203),
 (91, 46),
 (92, 246),
 (93, 101),
 (94, 320),
 (95, 8),
 (96, 8),
 (98, 24),
 (99, 19),
 (100, 9),
 (103, 16),
 (105, 439),
 (108, 4),
 (109, 17

In [143]:
sorted_station_id_2_swip_times = sorted(station_id_2_swip_times,key=lambda x:x[1],reverse=True)
sorted_station_id_2_swip_times

[(261, 3544),
 (759, 2193),
 (1414, 1959),
 (406, 1886),
 (349, 1807),
 (404, 1800),
 (806, 1489),
 (633, 1440),
 (795, 1405),
 (767, 1396),
 (387, 1384),
 (263, 1367),
 (1067, 1358),
 (1052, 1263),
 (610, 1193),
 (1173, 1187),
 (711, 1175),
 (1409, 1165),
 (555, 1120),
 (987, 1118),
 (1155, 1113),
 (28, 1081),
 (1011, 1075),
 (1208, 1071),
 (122, 1056),
 (553, 1023),
 (983, 993),
 (1120, 992),
 (508, 967),
 (534, 949),
 (850, 944),
 (419, 942),
 (216, 918),
 (87, 902),
 (1314, 897),
 (184, 889),
 (761, 870),
 (1358, 841),
 (447, 840),
 (1257, 840),
 (424, 812),
 (1063, 777),
 (88, 748),
 (913, 733),
 (354, 725),
 (787, 715),
 (31, 708),
 (189, 703),
 (550, 691),
 (405, 681),
 (128, 678),
 (908, 675),
 (744, 671),
 (320, 662),
 (807, 655),
 (1207, 654),
 (1056, 647),
 (758, 640),
 (899, 639),
 (1268, 633),
 (1140, 628),
 (430, 625),
 (403, 608),
 (1163, 604),
 (692, 592),
 (682, 585),
 (235, 580),
 (1009, 579),
 (1108, 575),
 (1018, 563),
 (1061, 560),
 (833, 548),
 (431, 547),
 (374, 

In [147]:
id_2_station['261']

'北京西路口'

In [ ]:
od_trips_groupby_id_hour.groupby('start_station_id')['nums'].transform('sum')

In [120]:
#'hour','minu','line','start_station_id','end_station_id','time'
#可以获取各个线路各个时刻在各个站点的刷卡数量
od_trips_groupby_line_hour_id = can_used_od_trips_dataframe.groupby(['line','hour','start_station_id' \
                                                                ]).agg({\
                                                                              'end_station_id':list, \
                                                                              'minu':list}).reset_index()
od_trips_groupby_line_hour_id

,line,hour,start_station_id,end_station_id,minu
0,1,05,806,"[1409, 430]","[50, 53]"
1,1,06,128,"[1414, 349, 1392, 261, 555, 633, 261, 261, 909...","[14, 45, 36, 14, 14, 15, 56, 56, 14, 15, 56]"
2,1,06,216,"[555, 261, 431, 555]","[37, 37, 57, 37]"
3,1,06,261,"[555, 555, 555, 555, 555, 828]","[52, 52, 22, 21, 52, 43]"
4,1,06,263,"[261, 261, 261]","[19, 19, 49]"
...,...,...,...,...,...
37460,4005,16,1056,[1056],[27]
37461,4005,16,1057,[373],[59]
37462,4005,16,1389,"[1389, 765, 1389, 263]","[32, 33, 33, 33]"
37463,4600,07,985,"[28, 31]","[57, 57]"


In [31]:

day_1001_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-10-01']
day_1001_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
377164,0x0C6B84DFFBEB7E3112481AE23499566B,222,7703,1,2023-10-01,08:33:23
377165,0x13A08316FB35BC0DFE0A98D00604E7E7,6,7744,4,2023-10-01,17:51:06
377166,0x1A146FC5EC5C78862B9A10C3004FCC2C,3,7477,15,2023-10-01,15:10:36
377167,0x3DC18BE83960BE730FC7CDCB33E1F2F5,226,7053,1,2023-10-01,15:25:31
377168,0x47602A7B7A8F237606D283798F822EEC,219,7158,21,2023-10-01,14:51:54
...,...,...,...,...,...,...
724622,0x6B5CCBC949D06967421DDEFFA8682697,13,7618,22,2023-10-01,11:09:08
724623,0x7FF2BF197AE5BB5ECCBC9BC64FFEE659,7,5020,10,2023-10-01,15:34:58
724624,0x89C88A5125FCAC7607C396EDABD8FA50,320,4451,9,2023-10-01,10:13:20
724625,0xA159FB0957C071A17B4FC41C6F622556,61,3826,10,2023-10-01,17:22:23


In [90]:
import heapq
# 要找出的最小的k个值
k = 3
# 使用列表推导式找出每行的最小的k个值
station_distance_k_values = [heapq.nsmallest(k, row) for row in station_pair_distance]

print(station_distance_k_values)

[[0.0, 321.81251849286264, 724.5687536525421], [0.0, 580.5780322709488, 682.3178681080469], [0.0, 176.91933282856345, 213.2404862913487], [0.0, 503.6789505716781, 534.4169974214412], [0.0, 209.05568164792004, 442.1250172769303], [0.0, 296.21522036957384, 383.51138986302936], [0.0, 4435088.475074348, 4435250.452118327], [0.0, 354.6858236746748, 555.1230557902543], [0.0, 14.886386699090393, 207.08484869433718], [0.0, 297.63156824634524, 636.3452050425832], [0.0, 398.0248514969028, 632.0148206737043], [0.0, 104.61891884172098, 133.632105665866], [0.0, 43.630723995853465, 104.61891884172098], [0.0, 43.630723995853465, 133.632105665866], [0.0, 286.60771430315674, 315.5605814385828], [0.0, 340.09463281064643, 355.93598799583236], [0.0, 281.3111787396845, 467.81530576598226], [0.0, 340.09463281064643, 432.8495548486716], [0.0, 232.25487924791304, 356.84214725062606], [0.0, 299.2912072045584, 317.6182000238952], [0.0, 1330.955616206553, 1380.397167007911], [0.0, 500.18443712070837, 608.2958651

In [102]:
# 将列表转换为NumPy数组
station_pair_distance_ndarray = np.array(station_pair_distance)
# 每行的最小的k个值所对应的index
k = 1  # 可以自定义k的值
station_pair_distance_min_k_indices = [np.argpartition(row, k)[:k] for row in station_pair_distance_ndarray]
print(station_pair_distance_min_k_indices)

[array([0]), array([1]), array([2]), array([3]), array([4]), array([5]), array([6]), array([7]), array([8]), array([9]), array([10]), array([11]), array([12]), array([13]), array([14]), array([15]), array([16]), array([17]), array([18]), array([19]), array([20]), array([21]), array([22]), array([23]), array([24]), array([25]), array([26]), array([27]), array([28]), array([29]), array([30]), array([31]), array([32]), array([33]), array([34]), array([35]), array([36]), array([37]), array([38]), array([39]), array([40]), array([41]), array([42]), array([43]), array([44]), array([45]), array([46]), array([47]), array([48]), array([49]), array([50]), array([51]), array([52]), array([53]), array([54]), array([55]), array([56]), array([57]), array([58]), array([59]), array([60]), array([61]), array([62]), array([63]), array([64]), array([65]), array([66]), array([67]), array([68]), array([69]), array([70]), array([71]), array([72]), array([73]), array([74]), array([75]), array([76]), array([7

In [105]:
# 找出每行最小的k个值所对应的索引
k=6
station_distance_min_indices = [sorted(range(len(row)), key=lambda i: row[i])[:k] for row in station_pair_distance]
station_distance_min_indices
# 按照值的大小对索引进行排序
#sorted_indices = sorted(range(len(data)), key=lambda i: data[i][min_indices[i][0]])

#print("按照最小值所对应的索引进行排序后的结果：", sorted_indices)

[[0, 536, 1259, 1226, 454, 977],
 [1, 361, 1225, 360, 362, 780],
 [2, 252, 1030, 704, 790, 15],
 [3, 381, 849, 925, 379, 380],
 [4, 1008, 265, 840, 841, 790],
 [5, 797, 222, 856, 855, 686],
 [6, 1384, 1183, 1383, 540, 906],
 [7, 637, 158, 1424, 17, 1425],
 [8, 206, 716, 164, 777, 468],
 [9, 1024, 259, 238, 304, 586],
 [10, 657, 1244, 1245, 1267, 324],
 [11, 12, 13, 462, 456, 870],
 [12, 13, 11, 462, 456, 870],
 [13, 12, 11, 462, 456, 870],
 [14, 915, 449, 69, 584, 447],
 [15, 17, 1030, 126, 1034, 252],
 [16, 1367, 1366, 755, 1358, 404],
 [17, 15, 637, 1030, 126, 252],
 [18, 578, 672, 674, 673, 19],
 [19, 673, 672, 674, 18, 578],
 [20, 103, 1202, 877, 593, 876],
 [21, 676, 853, 949, 855, 856],
 [22, 949, 1124, 1010, 1123, 853],
 [23, 1017, 745, 1018, 1073, 1074],
 [24, 625, 1339, 623, 1413, 353],
 [25, 710, 738, 658, 1139, 363],
 [26, 632, 916, 1188, 33, 1346],
 [27, 1278, 823, 824, 390, 1343],
 [28, 31, 594, 30, 29, 32],
 [29, 30, 375, 28, 31, 79],
 [30, 29, 375, 28, 31, 79],
 [31, 28,

# 任务
1.从各个站点的经纬度出发，计算出每个站点距离其他站点的距离，结果大概为1000*1000的方阵，然后我们一般会选择前10个，结果为1000*10


In [84]:
from geopy import distance
#(纬度，经度)
point1 = (26.633052,106.628469)
point2 = (26.635156,106.617741)
dist = distance.distance(point1,point2).m
print('距离',dist)

距离 1093.3717847368966


In [3]:
sheet1 = pd.read_csv('./card_info.csv')

In [4]:
sheet1

,card_identity,card_type,custom_time,line_identity,car_identity,consume,station_identity,data_src,data_load_time,partitionday
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,公交云卡,2023-09-29 08:59:40,28,7520,1.8,54,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
1,0x1FFC5D30D96AE2C4524842A09AA67C44,成人卡,2023-09-29 11:08:22,84,5085,1.8,24,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
2,0x65F9DAD54040FBE48972B10F3B6337DC,通卡学生卡,2023-09-29 08:51:28,25,6718,0.0,4,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
3,0x747A26B4727A9A3094A1881F29D01510,学生卡,2023-09-29 22:32:48,208,6655,1.0,1,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,公交云卡,2023-09-29 19:50:37,204,4472,1.8,46,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
...,...,...,...,...,...,...,...,...,...,...
724622,0x6B5CCBC949D06967421DDEFFA8682697,通卡学生卡,2023-10-01 11:09:08,13,7618,0.0,22,黔中市公共交通(集团)有限公司,2024-01-18 15:16:23,2023-10-01
724623,0x7FF2BF197AE5BB5ECCBC9BC64FFEE659,公交云卡,2023-10-01 15:34:58,7,5020,1.8,10,黔中市公共交通(集团)有限公司,2024-01-18 15:16:23,2023-10-01
724624,0x89C88A5125FCAC7607C396EDABD8FA50,成人卡,2023-10-01 10:13:20,320,4451,0.0,9,黔中市公共交通(集团)有限公司,2024-01-18 15:16:23,2023-10-01
724625,0xA159FB0957C071A17B4FC41C6F622556,通卡学生卡,2023-10-01 17:22:23,61,3826,0.0,10,黔中市公共交通(集团)有限公司,2024-01-18 15:16:23,2023-10-01


In [47]:
day_0929 = sheet1[sheet1['partitionday']=='2023-09-29']
day_0929

,card_identity,card_type,custom_time,line_identity,car_identity,consume,station_identity,data_src,data_load_time,partitionday
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,公交云卡,2023-09-29 08:59:40,28,7520,1.8,54,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
1,0x1FFC5D30D96AE2C4524842A09AA67C44,成人卡,2023-09-29 11:08:22,84,5085,1.8,24,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
2,0x65F9DAD54040FBE48972B10F3B6337DC,通卡学生卡,2023-09-29 08:51:28,25,6718,0.0,4,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
3,0x747A26B4727A9A3094A1881F29D01510,学生卡,2023-09-29 22:32:48,208,6655,1.0,1,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,公交云卡,2023-09-29 19:50:37,204,4472,1.8,46,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
...,...,...,...,...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,通卡学生卡,2023-09-29 08:48:38,52,5054,0.0,22,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,通卡学生卡,2023-09-29 10:33:45,1,4662,0.0,6,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377161,0x6D191CC173F9415C1148B434D2F9012E,通卡学生卡,2023-09-29 14:57:10,203,4810,0.0,5,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377162,0x7896A07D162530E734F876D16301EFEB,公交云卡,2023-09-29 15:08:49,73,5176,1.8,23,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29


In [28]:
#先找出每个人的od对需要做两件事：
#1.确定每个公交线路上每个上车站沿着这个线路可达的站
#2.确定每个站点与周围站点的距离。
#起点站好确定，但是终点站不好确定，看这条线路跟他下一趟车上车位置距离最近的站是哪个，这个站就是他这趟路程的终点站。
#

,card_identity,card_type,custom_time,line_identity,car_identity,consume,station_identity,data_src,data_load_time,partitionday
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,公交云卡,2023-09-29 08:59:40,28,7520,1.8,54,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
1,0x1FFC5D30D96AE2C4524842A09AA67C44,成人卡,2023-09-29 11:08:22,84,5085,1.8,24,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
2,0x65F9DAD54040FBE48972B10F3B6337DC,通卡学生卡,2023-09-29 08:51:28,25,6718,0.0,4,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
3,0x747A26B4727A9A3094A1881F29D01510,学生卡,2023-09-29 22:32:48,208,6655,1.0,1,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,公交云卡,2023-09-29 19:50:37,204,4472,1.8,46,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
...,...,...,...,...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,通卡学生卡,2023-09-29 08:48:38,52,5054,0.0,22,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,通卡学生卡,2023-09-29 10:33:45,1,4662,0.0,6,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377161,0x6D191CC173F9415C1148B434D2F9012E,通卡学生卡,2023-09-29 14:57:10,203,4810,0.0,5,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377162,0x7896A07D162530E734F876D16301EFEB,公交云卡,2023-09-29 15:08:49,73,5176,1.8,23,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29


In [48]:
#公交站点数据：
sheet_bus

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
0,1,1,1,火车站,2023-10-01,17
1,1,1,2,展览馆,2023-10-01,318
2,1,1,3,新路口,2023-10-01,510
3,1,1,4,邮电大楼,2023-10-01,678
4,1,1,5,大十字,2023-10-01,598
...,...,...,...,...,...,...
6475,4005,0,10,南垭路口,2023-10-01,2
6476,4600,1,7,粑粑坳,2023-10-01,2
6477,4601,1,14,白金大道中,2023-10-01,1
6478,4601,1,16,刚玉街,2023-10-01,1


In [29]:
day_0929_card_identity=day_0929.groupby('card_identity').agg({'custom_time':list,'line_identity':list,'station_identity':list}).reset_index()

In [38]:
for time in list(zip(day_0929_card_identity[3:4]['custom_time'].values[0],day_0929_card_identity[3:4]['line_identity'].values[0],day_0929_card_identity[3:4]['station_identity'].values[0])):
    print(time)

('2023-09-29 09:26:05', 207, 14)
('2023-09-29 19:24:18', 202, 39)
('2023-09-29 16:29:50', 5, 5)
('2023-09-29 09:20:51', 204, 8)


In [39]:
sheet_bus[sheet_bus['线路编号']==204]['站点名称'].values[0]
#上车明秀宾馆，中途学士路口，回来皂角井站，大西门站

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
3093,204,1,1,洛平公交枢纽,2023-10-01,693
3094,204,1,2,明珠大道,2023-10-01,127
3095,204,1,3,西坝行政中心,2023-10-01,60
3096,204,1,8,明秀宾馆,2023-10-01,326
3097,204,1,9,西坝公园站,2023-10-01,128
...,...,...,...,...,...,...
3149,204,0,62,民族大学,2023-10-01,14
3150,204,0,63,贵州大学轨道站,2023-10-01,53
3151,204,0,64,贵州大学,2023-10-01,5
3152,204,0,68,红阳厂,2023-10-01,16


In [41]:
sheet_bus[sheet_bus['站点名称']=='学士路口']

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
3040,203,1,25,学士路口,2023-10-01,2
3197,207,1,14,学士路口,2023-10-01,135
3219,207,0,38,学士路口,2023-10-01,49
6204,809,1,12,学士路口,2023-10-01,2


In [45]:
sheet_bus[sheet_bus['线路编号']==202][sheet_bus[sheet_bus['线路编号']==202]['站点名称']=='明秀宾馆']

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
2957,202,1,5,明秀宾馆,2023-10-01,44


In [46]:
sheet_bus[sheet_bus['线路编号']==202]

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
2953,202,1,1,洛平公交枢纽,2023-10-01,5
2954,202,1,2,迎宾路口,2023-10-01,172
2955,202,1,3,阳光水乡,2023-10-01,10
2956,202,1,4,贵筑路口,2023-10-01,84
2957,202,1,5,明秀宾馆,2023-10-01,44
...,...,...,...,...,...,...
3011,202,0,60,西坝公园站,2023-10-01,17
3012,202,0,61,贵筑路口,2023-10-01,11
3013,202,0,62,阳光水乡,2023-10-01,47
3014,202,0,63,迎宾路口,2023-10-01,124


In [5]:
sheet2 = pd.read_csv('./bus_swiping_card_info1.csv')

In [10]:
sheet2[sheet2['car_identity']==5011]

,car_identity,card_type,custom_time,line_identity,card_identity,station_identity,partitionday
0,5011,公交云卡,2024-03-16 08:09:27,7,0x000229AC214F69A81F266A521394C628,33,2024-03-18
451,5011,公交云卡,2024-03-20 14:07:43,7,0x0065D26071147DE0BC86C8E64983FC74,36,2024-03-22
1459,5011,成人卡,2024-03-18 07:04:23,7,0x0126BD3073CD16D2F2E70A8351EBF7C6,1,2024-03-20
2181,5011,员工卡,2024-03-17 22:11:06,7,0x01C1A5494079A4081C02B4A32B3DB12E,19,2024-03-19
2568,5011,公交云卡,2024-03-19 06:49:30,7,0x01FF9B8D32C9257DDF14D5E4C8793AB5,6,2024-03-21
...,...,...,...,...,...,...,...
678528,5011,公交云卡,2024-03-17 07:13:01,7,0xFCC1209616E73D34895E155D15DFB9BB,8,2024-03-19
680707,5011,公交云卡,2024-03-19 07:40:42,7,0xFE4E222E0A38B9DBF9C617CAA5328328,30,2024-03-21
681425,5011,公交云卡,2024-03-18 22:21:18,7,0xFF0C6D1D29C9B45BA194E3D0AF2A68B3,11,2024-03-20
682344,5011,通卡学生卡,2024-03-19 17:56:35,7,0xFFC6895409A569630FBD2CAEC8D8A9A4,5,2024-03-21


In [14]:
sheet2['day'] = sheet2['custom_time'].apply(lambda x:x.split(' ')[0])

In [17]:
car_5011 = sheet2[sheet2['car_identity']==5011]

In [18]:
car_5011[car_5011['day']=='2024-03-17']

,car_identity,card_type,custom_time,line_identity,card_identity,station_identity,partitionday,day
2181,5011,员工卡,2024-03-17 22:11:06,7,0x01C1A5494079A4081C02B4A32B3DB12E,19,2024-03-19,2024-03-17
3697,5011,公交云卡,2024-03-17 20:48:46,7,0x02C3A8E96F1E3A8FFE0E7C3899028979,40,2024-03-19,2024-03-17
6999,5011,通卡学生卡,2024-03-17 17:51:47,7,0x054A41D277056350639E729668A96BBC,32,2024-03-19,2024-03-17
7002,5011,通卡学生卡,2024-03-17 17:51:46,7,0x054A41D277056350639E729668A96BBC,32,2024-03-19,2024-03-17
7017,5011,通卡学生卡,2024-03-17 13:25:26,7,0x055129A027AB92B0C516C7424E1D5282,43,2024-03-19,2024-03-17
...,...,...,...,...,...,...,...,...
656380,5011,通卡学生卡,2024-03-17 08:57:41,7,0xEC0A35EAC83E1DF003377C4679A00115,1,2024-03-19,2024-03-17
663993,5011,通卡学生卡,2024-03-17 10:47:35,7,0xF19185DBFE7086FB8FD178FE58442BE9,43,2024-03-19,2024-03-17
667366,5011,公交云卡,2024-03-17 14:33:15,7,0xF401F351EF1A83192A0F93A1E7B22166,7,2024-03-19,2024-03-17
670054,5011,通卡学生卡,2024-03-17 12:59:57,7,0xF60E0B2D83925AE551D81D4CC4EFC1F2,37,2024-03-19,2024-03-17


In [23]:
sheet2[sheet2['card_identity']=='0xFCC1209616E73D34895E155D15DFB9BB']

,car_identity,card_type,custom_time,line_identity,card_identity,station_identity,partitionday,day
678521,7639,公交云卡,2024-03-18 17:18:43,33,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-20,2024-03-18
678522,4422,公交云卡,2024-03-20 07:09:45,49,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-22,2024-03-20
678523,7531,公交云卡,2024-03-20 17:21:12,33,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-22,2024-03-20
678524,5013,公交云卡,2024-03-19 17:11:32,7,0xFCC1209616E73D34895E155D15DFB9BB,33,2024-03-21,2024-03-19
678525,4422,公交云卡,2024-03-19 07:09:18,49,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-21,2024-03-19
678526,5011,公交云卡,2024-03-16 07:23:20,7,0xFCC1209616E73D34895E155D15DFB9BB,8,2024-03-18,2024-03-16
678527,7649,公交云卡,2024-03-16 16:51:48,33,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-18,2024-03-16
678528,5011,公交云卡,2024-03-17 07:13:01,7,0xFCC1209616E73D34895E155D15DFB9BB,8,2024-03-19,2024-03-17
678529,5016,公交云卡,2024-03-17 16:21:17,7,0xFCC1209616E73D34895E155D15DFB9BB,33,2024-03-19,2024-03-17
678530,4422,公交云卡,2024-03-18 07:01:38,49,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-20,2024-03-18


In [20]:
sheet_bus = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')

In [21]:
sheet_bus[sheet_bus['线路编号']==49]
#金顶路口	花果园金融街	花果园购物中心	花果园湿地公园	中山中路	遵义中路北
#龙洞堡客站

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
1643,49,1,1,阿哈湖湿地公园,2023-10-01,12
1644,49,1,2,延安南路口,2023-10-01,321
1645,49,1,3,公园中路南,2023-10-01,220
1646,49,1,4,公园中路,2023-10-01,203
1647,49,1,5,兰花广场,2023-10-01,74
1648,49,1,6,花果园大街,2023-10-01,116
1649,49,1,7,花果园大街中,2023-10-01,57
1650,49,1,8,花果园大街东,2023-10-01,131
1651,49,1,9,狮峰路口,2023-10-01,115
1652,49,1,10,浣沙桥,2023-10-01,97


In [22]:
sheet_bus[sheet_bus['线路编号']==33]
#金顶路口	花果园金融街	花果园购物中心	花果园湿地公园	中山中路	遵义中路北
#龙洞堡客站

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
1034,33,1,1,贵大茶店校区,2023-10-01,1
1035,33,1,2,中天花园路口,2023-10-01,8
1036,33,1,3,茶店,2023-10-01,120
1037,33,1,4,计质检所,2023-10-01,271
1038,33,1,5,省二医,2023-10-01,5
1039,33,1,6,大营坡,2023-10-01,154
1040,33,1,7,贵开路,2023-10-01,143
1041,33,1,8,黔中医学院,2023-10-01,90
1042,33,1,9,北京路,2023-10-01,52
1043,33,1,10,北京西路口,2023-10-01,74


In [ ]:
#早上乘坐49号线路在22号站贵开路上车
#晚上乘坐33号线路在22号站湘雅村上车
#湘雅村后面的站有：【花果园，花香村，浣沙桥，黔灵山公园，北京西路口，北京路，黔中医学院，贵开路，大营坡，省二医，计质检所】
#贵开路后面的站有：【贵乌路，红边门，友谊路，黔灵西路，浣沙桥，花香村，狮峰路口，花果园大街东，花果园购物中心，】

In [ ]:
#湘雅村最近的站为狮峰路口，所以极有可能狮峰路口下车，湘雅村回去。

In [24]:
#周六周日
sheet_bus[sheet_bus['线路编号']==7]
#工作日


,线路编号,方向,站点编号,站点名称,日期,刷卡数量
166,7,1,1,世纪园小区,2023-10-01,18
167,7,1,2,丽景阳天,2023-10-01,175
168,7,1,3,小石城二期,2023-10-01,202
169,7,1,4,小石城,2023-10-01,141
170,7,1,5,春雷水库,2023-10-01,77
171,7,1,6,沙河花园,2023-10-01,163
172,7,1,7,鹿冲关路,2023-10-01,217
173,7,1,8,贵开路,2023-10-01,183
174,7,1,9,贵州医科大学,2023-10-01,13
175,7,1,10,六广门,2023-10-01,91
